In [1]:
import numpy as np
import pandas as pd
from scipy import optimize

from gaus_vertex import *

In [2]:
# for n_mixs data points, want an array with cololmns for each monomer in mix, polymer densities, chi value, and phase identification

n_mixs = 1
M = 5 # number of monomers per polymer

N_m = 1# length of monomer
b = 1# kuhn length
n_p = 10# number of polymer types per mix

chi_array = np.arange(8, 25, 1)/(N_m*M)

# generate polymers and densities
rand_polys = np.random.choice(2,(n_mixs, n_p, M))
rand_dens = np.random.dirichlet(np.ones(n_p),size=(1, n_mixs))[0]

# assign phase to each mix
data = pd.DataFrame()

for i, mix in enumerate(rand_polys):
    if not np.all(mix == mix[0]): #in case all homopolymers (meaning no separation at all)
        dens = rand_dens[i]
        
        res = find_phase(mix, dens, N_m, b, M, chi_array) #returns an array of phase results at a range of chi values
        chis = res[::2]
        phase_names = res[1::2]
        for i in range(len(chis)):
#             row = np.append(mix.flatten(), [dens.flatten(), float(chis[i]), phase_names[i]])
            row = np.append(np.append(mix.flatten(), dens), [float(chis[i]), phase_names[i]])
            row = pd.DataFrame(row).T
            
            if len(data) == 0:
                data = row
            else:
                data = data.append(row)

# np.savetxt(r"TEST_data_M="+ str(M) +"_1e" + str(np.log10(n_mixs)), data, delimiter = ",")

# print("one done")

In [3]:
data

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,1.6,dis
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,1.8,dis
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,2.0,macro
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,2.2,macro
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,2.4,macro
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,2.6,macro
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,2.8,macro
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,3.0,macro
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,3.2,macro
0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0357005460084865,0.07535881384200546,0.3445782944808959,0.029591477024973244,0.07899813026909558,0.07449291074551526,0.1620036852531656,0.0045757213794032,3.4,macro


In [ ]:
#